# 第4章: 言語解析

問題30から問題35までは、以下の文章`text`（太宰治の『走れメロス』の冒頭部分）に対して、言語解析を実施せよ。問題36から問題39までは、国家を説明した文書群（日本語版ウィキペディア記事から抽出したテキスト群）をコーパスとして、言語解析を実施せよ。

In [1]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

In [ ]:
# !pip install mecab-python3
# !pip install pandas
# !pip install unidic
# !python -m unidic download
# !pip install japanize-matplotlib

## 30. 動詞
文章`text`に含まれる動詞をすべて表示せよ。

In [40]:
import MeCab
mecab = MeCab.Tagger()
parsed_text = mecab.parse(text)

In [44]:
parsed_text

'メロス\t名詞,普通名詞,一般,,,,メロス,メロス-melos,メロス,メロス,メロス,メロス,外,"","","","","","",体,メロス,メロス,メロス,メロス,"1","C1","",37122544834322944,135051\nは\t助詞,係助詞,,,,,ハ,は,は,ワ,は,ワ,和,"","","","","","",係助,ハ,ハ,ハ,ハ,"","動詞%F2@0,名詞%F1,形容詞%F2@-1","",8059703733133824,29321\n激怒\t名詞,普通名詞,サ変可能,,,,ゲキド,激怒,激怒,ゲキド,激怒,ゲキド,漢,"","","","","","",体,ゲキド,ゲキド,ゲキド,ゲキド,"1","C1","",3167426745344512,11523\nし\t動詞,非自立可能,,,サ行変格,連用形-一般,スル,為る,し,シ,する,スル,和,"","","","","","",用,シ,スル,シ,スル,"0","C5","",5370298291593857,19537\nた\t助動詞,,,,助動詞-タ,終止形-一般,タ,た,た,タ,た,タ,和,"","","","","","",助動,タ,タ,タ,タ,"","動詞%F2@1,形容詞%F4@-2","",5948916285711019,21642\n。\t補助記号,句点,,,,,,。,。,,。,,記号,"","","","","","",補助,,,,,"","","",6880571302400,25\n必ず\t副詞,,,,,,カナラズ,必ず,必ず,カナラズ,必ず,カナラズ,和,"","","","","","",相,カナラズ,カナラズ,カナラズ,カナラズ,"0","","",1899964749980160,6912\n、\t補助記号,読点,,,,,,、,、,,、,,記号,"","","","","","",補助,,,,,"","","",6605693395456,24\nかの\t連体詞,,,,,,カノ,彼の,かの,カノ,かの,カノ,和,"","","","","","",相,カノ,カノ,カノ,カノ,"1","","",1914533245493760,6965\n邪智\t名詞,普通名詞,一般,,,,ジャチ,邪知,邪智,ジャチ,邪智,ジャチ,漢,"","","","",

In [29]:
for line in parsed_text:
    if line == 'EOS\n':
        pass
    else:
        tab_splitted_line = line.split('\t') # 見出し語と解析結果を分割
        comma_splitted_line = tab_splitted_line[1].split(',') # 解析結果を分割
        print(comma_splitted_line[0]) # 品詞（pos）

名詞
副詞
名詞
名詞
名詞
助詞


## 31. 動詞の原型
文章`text`に含まれる動詞と、その原型をすべて表示せよ。

In [ ]:
verb_dict = {}
for neko_list in neko_general_list:
    for neko_dict in neko_list:
        if neko_dict['pos'] == '動詞':
            # 辞書からキーで値を取得し、キーが存在しない場合は None を返す
            verb_value = verb_dict.get(neko_dict['base'])
            if verb_value == None:
                # 始めて追加する場合にはカウントを1
                verb_dict[neko_dict['base']] = 1
            else:
                # 二回目以降に登場する場合にはカウントアップ
                verb_dict[neko_dict['base']] = verb_dict[neko_dict['base']] + 1

## 32. 「AのB」
文章`text`において、2つの名詞が「の」で連結されている名詞句をすべて抽出せよ。

## 33. 係り受け解析

文章`text`に係り受け解析を適用し、係り元と係り先のトークン（形態素や文節などの単位）をタブ区切り形式ですべて抽出せよ。

## 34. 主述の関係
文章`text`において、「メロス」が主語であるときの述語を抽出せよ。

## 35. 係り受け木
「メロスは激怒した。」の係り受け木を可視化せよ。

## 36. 単語の出現頻度

問題36から39までは、Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](/data/jawiki-country.json.gz)をコーパスと見なし、統計的な分析を行う。

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が"title"キーに、記事本文が"text"キーの辞書オブジェクトに格納され、そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される

まず、第3章の処理内容を参考に、Wikipedia記事からマークアップを除去し、各記事のテキストを抽出せよ。そして、コーパスにおける単語（形態素）の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 37. 名詞の出現頻度
コーパスにおける名詞の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

## 38. TF・IDF
日本に関する記事における名詞のTF・IDFスコアを求め、TF・IDFスコア上位20語とそのTF, IDF, TF・IDFを表示せよ。

## 39. Zipfの法則
コーパスにおける単語の出現頻度順位を横軸、その出現頻度を縦軸として、両対数グラフをプロットせよ。